<a href="https://colab.research.google.com/github/volcan4/ia-pipeline-etl-python/blob/main/BootcampSantander2023_com_SantanderDevWeek2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bootcamp Santander 2023 (Ciência de Dados Com Python) com Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [ ]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
bcs2023_api_url = 'https://sdw-2023-prd.up.railway.app/users/'

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [ ]:
import pandas as pd

df = pd.read_csv('BCS2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[3183, 3187, 3188]


In [ ]:
import requests
import json

def get_user(id):
  response = requests.get(f'{bcs2023_api_url}{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 3183,
    "name": "Luffy",
    "account": {
      "id": 3363,
      "number": "01071-1",
      "agency": "0001",
      "balance": 0.0,
      "limit": 30000000.0
    },
    "card": {
      "id": 3079,
      "number": "**** **** **** 1076",
      "limit": 3000000000.0
    },
    "features": [],
    "news": [
      {
        "id": 6636,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Luffy, os tesouros das tuas aventuras s\u00e3o valiosos! Invista sabiamente para garantir um futuro sustent\u00e1vel e explorar novas oportunidades na \u00faltima ilha. Seu poder n\u00e3o est\u00e1 apenas em conquistar desafios, mas tamb\u00e9m em prosperar ao cuidar do seu tesouro!"
      },
      {
        "id": 6637,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Luffy, invista seus tesouros como um verdadeiro Rei dos Piratas! Fa\u00e

## **T**ransform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.9 MB/s eta 0:00:00


In [ ]:
# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Substitua o texto TODO por sua API Key da OpenAI, ela será salva como uma variável de ambiente.
openai_api_key = 'SuACHavEaPIaquI'

In [ ]:
import openai

openai.api_key = openai_api_key

def generate_ai_news(user):
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em markting bancário para os piratas do mundo de One Piece."
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos dos tesouros acumulados nas aventuras rumo a última ilha (máximo de 300 caracteres)"
      }
    ]
  )
  return completion.choices[0].message.content.strip('\"')

for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

Luffy, o rei dos piratas! Sua coragem te trouxe até aqui. Invista seus tesouros melhorando sua tripulação, navio e armas para a última ilha. Proteja seu futuro e garanta sua grandeza! #PirateInvesting
Zoro, valorize cada saque com estratégia. Invista seus tesouros com sabedoria na busca pela última ilha. Conquiste seu futuro com adaptabilidade. Rumo à grandiosidade! #PirateInvestments
Sanji, invista seus tesouros com sabedoria na última ilha. Garanta um futuro próspero e sustentável para a tripulação e desfrute das recompensas de suas aventuras. Seja o capitão financeiro que todos precisam! #InvestirÉPiratariaEstratégica


## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [ ]:
def update_user(user):
  response = requests.put(f"{bcs2023_api_url}{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Luffy updated? True!
User Zoro updated? True!
User Sanji updated? True!
